In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\Desktop\\YOLO-Custom-Object-Detection-Web-App-in-Python\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dell\\Desktop\\YOLO-Custom-Object-Detection-Web-App-in-Python'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class YoloDataValidationConfig:
    root_dir: Path
    status_file: str
    images_dir: Path
    annotations_dir: Path
    classes_file: Path  # Path to a file containing class labels

In [6]:
from src.ppeDetection.constants import *
from src.ppeDetection.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_yolo_data_validation_config(self) -> YoloDataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        yolo_data_validation_config = YoloDataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.STATUS_FILE,
            images_dir=config.images_dir,
            annotations_dir=config.annotations_dir,
            classes_file=config.classes_file,
        )

        return yolo_data_validation_config

In [9]:
class YoloDataValidation:
    def __init__(self, config: YoloDataValidationConfig):
        self.config = config

    def validate_classes(self) -> bool:
        """Validate that the classes file exists and has valid entries."""
        if not self.config.classes_file.exists():
            raise FileNotFoundError(f"Classes file not found: {self.config.classes_file}")
        
        with open(self.config.classes_file, 'r') as file:
            classes = [line.strip() for line in file.readlines()]
        
        if not classes:
            raise ValueError("Classes file is empty.")
        
        print("Classes validation passed.")
        return True

    def validate_annotations(self) -> bool:
        """Validate that all annotation files match the image files and have correct YOLO format."""
        annotations = list(self.config.annotations_dir.glob("*.txt"))
        images = list(self.config.images_dir.glob("*.*"))

        if len(images) == 0:
            raise ValueError("No images found in the directory.")

        if len(annotations) == 0:
            raise ValueError("No annotation files found in the directory.")
        
        for image_file in images:
            annotation_file = self.config.annotations_dir / f"{image_file.stem}.txt"
            if not annotation_file.exists():
                raise FileNotFoundError(f"Missing annotation file for image: {image_file}")
            
            with open(annotation_file, 'r') as file:
                for line in file.readlines():
                    parts = line.strip().split()
                    if len(parts) < 5:
                        raise ValueError(f"Invalid annotation format in {annotation_file}: {line}")
        
        print("Annotation validation passed.")
        return True

    def validate_all(self) -> bool:
        """Run all validations."""
        try:
            classes_valid = self.validate_classes()
            annotations_valid = self.validate_annotations()

            validation_status = classes_valid and annotations_valid

            with open(self.config.status_file, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            print("Data validation completed successfully.")
            return validation_status

        except Exception as e:
            with open(self.config.status_file, 'w') as f:
                f.write(f"Validation status: False\nError: {str(e)}")
            raise e